In [ ]:
# Cell 1: Install Required Packages
import subprocess
import sys
import os

# Use C:\pylib to avoid Windows long path issues
PYLIB_PATH = r"C:\pylib"
os.makedirs(PYLIB_PATH, exist_ok=True)

if PYLIB_PATH not in sys.path:
    sys.path.insert(0, PYLIB_PATH)

packages = [
    "azure-identity",
    "azure-mgmt-resource",
    "azure-mgmt-keyvault",
    "azure-mgmt-containerinstance",
    "azure-mgmt-authorization",
    "requests",
    "cryptography"
]

print("📦 Installing required packages...")
for pkg in packages:
    print(f"  Installing {pkg}...", end=" ")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", pkg,
         "--target", PYLIB_PATH, "--quiet", "--upgrade"],
        capture_output=True, text=True
    )
    print("✅" if result.returncode == 0 else "❌")

print("\n✅ Package installation complete!")

## 🔧 Configuration

Set your deployment parameters below. Change `INDEX` to create separate deployments.

In [ ]:
# Cell 2: Configuration
import os
import json
import subprocess
from dataclasses import dataclass
from typing import Optional

# ╔════════════════════════════════════════════════════════════════════╗
# ║  🔧 CHANGE INDEX TO CREATE NEW DEPLOYMENT                          ║
# ╚════════════════════════════════════════════════════════════════════╝
INDEX = 1  # <-- Change this for new deployments

@dataclass
class Config:
    INDEX: int = INDEX
    WORKLOAD: str = "kvtls"
    ENVIRONMENT: str = "dev"
    LOCATION: str = "westus3"
    KEYVAULT_SKU: str = "standard"  # 'standard' or 'premium'
    
    # Auto-detected from Azure CLI
    SUBSCRIPTION_ID: Optional[str] = None
    TENANT_ID: Optional[str] = None
    ADMIN_OBJECT_ID: Optional[str] = None
    
    @property
    def index_str(self) -> str:
        return f"{self.INDEX:03d}"
    
    @property
    def resource_group_name(self) -> str:
        return f"rg-{self.WORKLOAD}-{self.ENVIRONMENT}-{self.index_str}"
    
    @property
    def keyvault_name(self) -> str:
        name = f"kv-{self.WORKLOAD}-{self.ENVIRONMENT}-{self.index_str}"
        return name[:24]
    
    @property
    def managed_identity_name(self) -> str:
        return f"id-{self.WORKLOAD}-{self.ENVIRONMENT}-{self.index_str}"

config = Config()

# Get Azure account info
def run_az(cmd: list) -> dict:
    result = subprocess.run(["az"] + cmd + ["-o", "json"], capture_output=True, text=True, shell=True)
    if result.returncode != 0:
        raise Exception(f"Azure CLI error: {result.stderr}")
    return json.loads(result.stdout) if result.stdout else {}

try:
    account = run_az(["account", "show"])
    config.SUBSCRIPTION_ID = account["id"]
    config.TENANT_ID = account["tenantId"]
    
    # Get current user's object ID
    user = run_az(["ad", "signed-in-user", "show"])
    config.ADMIN_OBJECT_ID = user["id"]
    
    print("✅ Azure CLI Configuration")
    print("=" * 50)
    print(f"  Subscription: {account['name']}")
    print(f"  Tenant ID:    {config.TENANT_ID}")
    print(f"  Admin OID:    {config.ADMIN_OBJECT_ID}")
    print()
    print("📋 Deployment Configuration")
    print("-" * 50)
    print(f"  Index:           {config.INDEX}")
    print(f"  Resource Group:  {config.resource_group_name}")
    print(f"  Key Vault:       {config.keyvault_name}")
    print(f"  Location:        {config.LOCATION}")
    print(f"  SKU:             {config.KEYVAULT_SKU}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Run 'az login' first!")

## 📦 Step 1: Create Resource Group

In [ ]:
# Cell 3: Create Resource Group
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

credential = DefaultAzureCredential()
resource_client = ResourceManagementClient(credential, config.SUBSCRIPTION_ID)

print(f"📦 Creating Resource Group: {config.resource_group_name}")

rg_result = resource_client.resource_groups.create_or_update(
    config.resource_group_name,
    {
        "location": config.LOCATION,
        "tags": {
            "Environment": config.ENVIRONMENT,
            "Workload": config.WORKLOAD,
            "Purpose": "KeyVault-Keyless-TLS"
        }
    }
)

print(f"✅ Resource Group created: {rg_result.name}")
print(f"   Location: {rg_result.location}")

## 🔑 Step 2: Create Key Vault

In [ ]:
# Cell 4: Create Key Vault with RBAC
from azure.mgmt.keyvault import KeyVaultManagementClient
from azure.mgmt.keyvault.models import (
    VaultCreateOrUpdateParameters,
    VaultProperties,
    Sku,
    SkuName,
    SkuFamily
)

kv_client = KeyVaultManagementClient(credential, config.SUBSCRIPTION_ID)

print(f"🔑 Creating Key Vault: {config.keyvault_name}")

sku_name = SkuName.PREMIUM if config.KEYVAULT_SKU == "premium" else SkuName.STANDARD

vault = kv_client.vaults.begin_create_or_update(
    config.resource_group_name,
    config.keyvault_name,
    VaultCreateOrUpdateParameters(
        location=config.LOCATION,
        tags={"Purpose": "Keyless-TLS"},
        properties=VaultProperties(
            tenant_id=config.TENANT_ID,
            sku=Sku(family=SkuFamily.A, name=sku_name),
            enable_rbac_authorization=True,
            enable_soft_delete=True,
            soft_delete_retention_in_days=7,
            enable_purge_protection=False,
            public_network_access="Enabled"
        )
    )
).result()

print(f"✅ Key Vault created!")
print(f"   Name: {vault.name}")
print(f"   URI:  {vault.properties.vault_uri}")
print(f"   SKU:  {vault.properties.sku.name}")

## 👤 Step 3: Assign Key Vault Administrator Role

In [ ]:
# Cell 5: Assign Key Vault Administrator Role to current user
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import RoleAssignmentCreateParameters
import uuid

auth_client = AuthorizationManagementClient(credential, config.SUBSCRIPTION_ID)

# Key Vault Administrator role ID
KV_ADMIN_ROLE = "00482a5a-887f-4fb3-b363-3b7fe8e74483"
role_def_id = f"/subscriptions/{config.SUBSCRIPTION_ID}/providers/Microsoft.Authorization/roleDefinitions/{KV_ADMIN_ROLE}"

print(f"👤 Assigning Key Vault Administrator role to current user...")

try:
    assignment = auth_client.role_assignments.create(
        scope=vault.id,
        role_assignment_name=str(uuid.uuid4()),
        parameters=RoleAssignmentCreateParameters(
            role_definition_id=role_def_id,
            principal_id=config.ADMIN_OBJECT_ID,
            principal_type="User"
        )
    )
    print(f"✅ Role assigned successfully!")
except Exception as e:
    if "RoleAssignmentExists" in str(e):
        print(f"✅ Role already assigned")
    else:
        print(f"⚠️ Role assignment: {e}")

print("\n⏳ Waiting 30s for RBAC to propagate...")
import time
time.sleep(30)
print("✅ Ready to create keys!")

## 🔐 Step 4: Create RSA and EC Keys

In [ ]:
# Cell 6: Create RSA and EC keys in Key Vault
from azure.keyvault.keys import KeyClient, KeyType, KeyCurveName

key_client = KeyClient(vault_url=vault.properties.vault_uri, credential=credential)

RSA_KEY_NAME = "rsa-tls-key"
EC_KEY_NAME = "ec-tls-key"

print("🔐 Creating cryptographic keys...")
print()

# Create RSA key (3072-bit for TLS)
print(f"  Creating RSA key: {RSA_KEY_NAME}")
rsa_key = key_client.create_rsa_key(
    name=RSA_KEY_NAME,
    size=3072,
    key_operations=["sign", "verify"],
    tags={"purpose": "tls-signing", "type": "rsa"}
)
print(f"  ✅ RSA key created: {rsa_key.name} ({rsa_key.key_type})")
print(f"     Key ID: {rsa_key.id}")
print()

# Create EC key (P-256 for TLS)
print(f"  Creating EC key: {EC_KEY_NAME}")
ec_key = key_client.create_ec_key(
    name=EC_KEY_NAME,
    curve=KeyCurveName.p_256,
    key_operations=["sign", "verify"],
    tags={"purpose": "tls-signing", "type": "ec"}
)
print(f"  ✅ EC key created: {ec_key.name} ({ec_key.key_type})")
print(f"     Key ID: {ec_key.id}")
print(f"     Curve: P-256")
print()

print("="*60)
print("🎯 Key URIs for OpenSSL Provider:")
print("-"*60)
print(f"  RSA: keyvault:{config.keyvault_name}:{RSA_KEY_NAME}")
print(f"  EC:  keyvault:{config.keyvault_name}:{EC_KEY_NAME}")
print()
print("Or using key-value format:")
print(f"  RSA: akv:type=keyvault,vault={config.keyvault_name},name={RSA_KEY_NAME}")
print(f"  EC:  akv:type=keyvault,vault={config.keyvault_name},name={EC_KEY_NAME}")

## 📜 Step 5: Generate Self-Signed Certificates

Create self-signed certificates using the keys in Key Vault.
These will be used for TLS on the nginx server.

In [ ]:
# Cell 7: Generate Self-Signed Certificates
# Note: For production, use a proper CA. This is for testing only.

from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, ec
from cryptography.hazmat.backends import default_backend
import datetime
import base64

# For self-signed certs, we need to sign locally since Key Vault
# doesn't support CSR signing directly. We'll create local keys
# for the certificate, but in production the nginx provider
# will use the Key Vault keys for TLS operations.

def create_self_signed_cert(subject_name: str, key_type: str = "rsa") -> tuple:
    """Create a self-signed certificate for testing."""
    
    if key_type == "rsa":
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=3072,
            backend=default_backend()
        )
    else:
        private_key = ec.generate_private_key(
            ec.SECP256R1(),
            backend=default_backend()
        )
    
    subject = issuer = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, "US"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Washington"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, "Redmond"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, "KeyVault Test"),
        x509.NameAttribute(NameOID.COMMON_NAME, subject_name),
    ])
    
    cert = (
        x509.CertificateBuilder()
        .subject_name(subject)
        .issuer_name(issuer)
        .public_key(private_key.public_key())
        .serial_number(x509.random_serial_number())
        .not_valid_before(datetime.datetime.utcnow())
        .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
        .add_extension(
            x509.SubjectAlternativeName([
                x509.DNSName("localhost"),
                x509.DNSName(subject_name),
            ]),
            critical=False,
        )
        .sign(private_key, hashes.SHA256(), default_backend())
    )
    
    cert_pem = cert.public_bytes(serialization.Encoding.PEM)
    key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    return cert_pem, key_pem

print("📜 Generating self-signed certificates...")
print()

# Create output directory
import os
cert_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "certs")
os.makedirs(cert_dir, exist_ok=True)

# RSA certificate
rsa_cert, rsa_key = create_self_signed_cert(f"rsa.{config.keyvault_name}.local", "rsa")
with open(os.path.join(cert_dir, "rsa-server.crt"), "wb") as f:
    f.write(rsa_cert)
with open(os.path.join(cert_dir, "rsa-server.key"), "wb") as f:
    f.write(rsa_key)
print(f"  ✅ RSA certificate: {cert_dir}/rsa-server.crt")

# EC certificate  
ec_cert, ec_key = create_self_signed_cert(f"ec.{config.keyvault_name}.local", "ec")
with open(os.path.join(cert_dir, "ec-server.crt"), "wb") as f:
    f.write(ec_cert)
with open(os.path.join(cert_dir, "ec-server.key"), "wb") as f:
    f.write(ec_key)
print(f"  ✅ EC certificate:  {cert_dir}/ec-server.crt")

print()
print(f"📁 Certificates saved to: {cert_dir}")
print()
print("⚠️  NOTE: For keyless TLS testing with the provider:")
print("    The nginx config should use the Key Vault URI instead of key files:")
print(f"    ssl_certificate_key \"store:keyvault:{config.keyvault_name}:{RSA_KEY_NAME}\";")

## 🧪 Step 6: Test Key Vault Access

Verify we can access the keys for signing operations.

In [ ]:
# Cell 8: Test Key Vault Key Access
from azure.keyvault.keys.crypto import CryptographyClient, SignatureAlgorithm
import hashlib

print("🧪 Testing Key Vault signing operations...")
print()

# Test data
test_data = b"Hello, Keyless TLS!"
digest = hashlib.sha256(test_data).digest()

# Test RSA signing
print(f"  Testing RSA key ({RSA_KEY_NAME})...")
rsa_crypto = CryptographyClient(rsa_key.id, credential=credential)
rsa_result = rsa_crypto.sign(SignatureAlgorithm.rs256, digest)
print(f"  ✅ RSA signature: {len(rsa_result.signature)} bytes")

# Verify RSA signature
rsa_verify = rsa_crypto.verify(SignatureAlgorithm.rs256, digest, rsa_result.signature)
print(f"  ✅ RSA verification: {rsa_verify.is_valid}")
print()

# Test EC signing
print(f"  Testing EC key ({EC_KEY_NAME})...")
ec_crypto = CryptographyClient(ec_key.id, credential=credential)
ec_result = ec_crypto.sign(SignatureAlgorithm.es256, digest)
print(f"  ✅ EC signature: {len(ec_result.signature)} bytes")

# Verify EC signature
ec_verify = ec_crypto.verify(SignatureAlgorithm.es256, digest, ec_result.signature)
print(f"  ✅ EC verification: {ec_verify.is_valid}")
print()

print("="*60)
print("✅ All Key Vault operations successful!")
print("   Keys are ready for keyless TLS with nginx.")

## 📋 Step 7: Generate nginx Configuration

Create nginx configuration that uses Key Vault keys via the OpenSSL provider.

In [ ]:
# Cell 9: Generate nginx and OpenSSL Provider Configuration
import os

config_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "nginx-config")
os.makedirs(config_dir, exist_ok=True)

# OpenSSL provider configuration
openssl_cnf = f"""# OpenSSL configuration for Azure Key Vault Provider
openssl_conf = openssl_init

[openssl_init]
providers = provider_section

[provider_section]
default = default_section
base = base_section
akv_provider = akv_provider_section

[default_section]
activate = 1

[base_section]
activate = 1

[akv_provider_section]
# Path to the AKV provider library
module = /opt/akv-provider/akv_provider.so
activate = 1
"""

# nginx configuration for Key Vault keyless TLS
nginx_conf = f"""# nginx configuration for Azure Key Vault Keyless TLS
# Uses the AKV OpenSSL Provider for private key operations

worker_processes auto;
error_log /var/log/nginx/error.log debug;
pid /run/nginx.pid;

events {{
    worker_connections 1024;
}}

http {{
    include /etc/nginx/mime.types;
    default_type application/octet-stream;
    
    log_format main '$remote_addr - $remote_user [$time_local] "$request" '
                    '$status $body_bytes_sent "$http_referer" '
                    '"$http_user_agent" ssl_protocol=$ssl_protocol '
                    'ssl_cipher=$ssl_cipher';
    
    access_log /var/log/nginx/access.log main;
    
    # HTTP server (redirect to HTTPS)
    server {{
        listen 80;
        server_name _;
        return 301 https://$host$request_uri;
    }}
    
    # RSA TLS server (port 8443)
    server {{
        listen 8443 ssl;
        server_name localhost;
        
        # Certificate (public)
        ssl_certificate /etc/nginx/certs/rsa-server.crt;
        
        # Key from Azure Key Vault (keyless!)
        ssl_certificate_key "store:keyvault:{config.keyvault_name}:{RSA_KEY_NAME}";
        
        ssl_protocols TLSv1.2 TLSv1.3;
        ssl_prefer_server_ciphers on;
        ssl_ciphers 'ECDHE-RSA-AES256-GCM-SHA384:ECDHE-RSA-AES128-GCM-SHA256';
        
        location / {{
            return 200 'RSA Keyless TLS with Azure Key Vault!\n';
            add_header Content-Type text/plain;
        }}
        
        location /health {{
            return 200 'OK';
            add_header Content-Type text/plain;
        }}
    }}
    
    # EC TLS server (port 8444)
    server {{
        listen 8444 ssl;
        server_name localhost;
        
        # Certificate (public)
        ssl_certificate /etc/nginx/certs/ec-server.crt;
        
        # Key from Azure Key Vault (keyless!)
        ssl_certificate_key "store:keyvault:{config.keyvault_name}:{EC_KEY_NAME}";
        
        ssl_protocols TLSv1.2 TLSv1.3;
        ssl_prefer_server_ciphers on;
        ssl_ciphers 'ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES128-GCM-SHA256';
        
        location / {{
            return 200 'EC Keyless TLS with Azure Key Vault!\n';
            add_header Content-Type text/plain;
        }}
        
        location /health {{
            return 200 'OK';
            add_header Content-Type text/plain;
        }}
    }}
}}
"""

# Write configurations
with open(os.path.join(config_dir, "openssl-provider.cnf"), "w") as f:
    f.write(openssl_cnf)
print(f"✅ OpenSSL config: {config_dir}/openssl-provider.cnf")

with open(os.path.join(config_dir, "nginx.conf"), "w") as f:
    f.write(nginx_conf)
print(f"✅ nginx config:   {config_dir}/nginx.conf")

print()
print("📋 nginx Key URI Format:")
print("-" * 50)
print(f"  RSA: store:keyvault:{config.keyvault_name}:{RSA_KEY_NAME}")
print(f"  EC:  store:keyvault:{config.keyvault_name}:{EC_KEY_NAME}")

## 📊 Deployment Summary

In [ ]:
# Cell 10: Deployment Summary
print("="*70)
print("  🎉 AZURE KEY VAULT KEYLESS TLS DEPLOYMENT COMPLETE")
print("="*70)
print()
print("📦 Resources Created:")
print("-"*70)
print(f"  Resource Group:  {config.resource_group_name}")
print(f"  Key Vault:       {config.keyvault_name}")
print(f"  Key Vault URI:   {vault.properties.vault_uri}")
print()
print("🔐 Keys Created:")
print("-"*70)
print(f"  RSA Key: {RSA_KEY_NAME} (3072-bit)")
print(f"  EC Key:  {EC_KEY_NAME} (P-256)")
print()
print("🔗 OpenSSL Provider URIs:")
print("-"*70)
print(f"  RSA: keyvault:{config.keyvault_name}:{RSA_KEY_NAME}")
print(f"  EC:  keyvault:{config.keyvault_name}:{EC_KEY_NAME}")
print()
print("  Alternative format:")
print(f"  RSA: kv:{config.keyvault_name}:{RSA_KEY_NAME}")
print(f"  EC:  akv:type=keyvault,vault={config.keyvault_name},name={EC_KEY_NAME}")
print()
print("🔑 Get Access Token:")
print("-"*70)
print("  # For Key Vault (different from Managed HSM!)")
print("  export AZURE_CLI_ACCESS_TOKEN=$(az account get-access-token \\")
print("      --resource https://vault.azure.net \\")
print("      --query accessToken -o tsv)")
print()
print("📁 Generated Files:")
print("-"*70)
print(f"  Certificates: {cert_dir}/")
print(f"  nginx Config: {config_dir}/")
print()
print("🧪 Next Steps:")
print("-"*70)
print("  1. Build the Rust provider: cd src_provider_rust && cargo build --release")
print("  2. Set OPENSSL_CONF to point to openssl-provider.cnf")
print("  3. Run nginx with the provider loaded")
print("  4. Test: curl -k https://localhost:8443/")
print()

---
## 🗑️ Cleanup

Run the cells below to delete all resources when you're done testing.

In [ ]:
# Cell 11: Cleanup - DELETE ALL RESOURCES
# ⚠️ WARNING: This will delete the resource group and ALL resources in it!

CONFIRM_DELETE = False  # Set to True to enable deletion

if CONFIRM_DELETE:
    print(f"🗑️ Deleting resource group: {config.resource_group_name}")
    print("   This will delete ALL resources including Key Vault and keys!")
    print()
    
    delete_operation = resource_client.resource_groups.begin_delete(
        config.resource_group_name
    )
    
    print("   ⏳ Deletion in progress (this may take a few minutes)...")
    delete_operation.wait()
    
    print(f"   ✅ Resource group '{config.resource_group_name}' deleted!")
else:
    print("⚠️ Cleanup skipped. Set CONFIRM_DELETE = True to delete resources.")
    print(f"   Resource Group: {config.resource_group_name}")
    print()
    print("   To delete manually:")
    print(f"   az group delete --name {config.resource_group_name} --yes")